In [1]:
import sys
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
sys.path.append("..")
from evaluation.generated_dataset import load_all_from_config
from evaluation.statistical_evaluator import StatisticalEvaluator
from evaluation.novelty import NoveltyFilter

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


In [2]:
datasets = {
    "WyFormer": ("WyckoffTransformer", "DiffCSP++10k"),
    "FlowMM": ("FlowMM",),
    "MiAD": ("MiAD", "CHGNet_free"),
    "DiffCSP++": ("DiffCSP++",),
    "DiffCSP": ("DiffCSP",),
    "SymmCD": ("SymmCD",),
    "MP-20-train": ("split", "train"),
    "MP-20-test": ("split", "test"),
    "MP-20-val": ("split", "val")
}

In [3]:
all_datasets = load_all_from_config(datasets=datasets.values(), dataset_name="mp_20")

In [4]:
test_evaluator = StatisticalEvaluator(all_datasets[('split', 'test')].data)

In [5]:
table = pd.DataFrame(
    index=datasets.keys(), columns=[
        "P1 (%)",
        "Space Group chi^2",
        "N atoms EMD"])
for name, transformations in tqdm(datasets.items()):
    dataset = all_datasets[transformations].data
    table.loc[name, "P1 (%)"] = 100 * (dataset.group == 1).mean()
    table.loc[name, r"Space Group chi^2"] = test_evaluator.get_sg_chi2(dataset)
    table.loc[name, "N atoms EMD"] = test_evaluator.get_num_atoms_emd(dataset)

  0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
table

,P1 (%),Space Group chi^2,N atoms EMD
WyFormer,1.880188,0.038074,8.821859
FlowMM,38.09762,11.132666,0.111229
MiAD,17.386955,3.368287,1.461373
DiffCSP++,1.64,0.041517,8.644447
DiffCSP,28.35,5.463993,0.136238
SymmCD,1.973615,0.046386,7.897631
MP-20-train,1.732017,0.025769,0.125059
MP-20-test,1.812956,0.0,0.0
MP-20-val,1.624848,0.03528,0.076691


In [7]:
novelty_reference = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
novelty_filter = NoveltyFilter(novelty_reference)

In [8]:
table = pd.DataFrame(
    index=datasets.keys(), columns=[
        "P1 (%)",
        "Space Group chi^2",
        "N atoms EMD"])
for name, transformations in tqdm(datasets.items()):
    dataset = all_datasets[transformations].data
    dataset["is_novel"] = dataset.apply(novelty_filter.is_novel, axis=1)
    dataset["n_atoms"] = dataset.structure.map(len)
    novel = dataset[dataset["is_novel"]]
    table.loc[name, "P1 (%)"] = 100 * (novel.group == 1).mean()
    table.loc[name, r"Space Group chi^2"] = test_evaluator.get_sg_chi2(novel)
    if len(novel) > 0:
        table.loc[name, "N atoms EMD"] = test_evaluator.get_num_atoms_emd(novel)
    dataset.drop(columns="cdvae_crystal").to_pickle(Path("data_export", name + ".pkl.gz"))

  0%|          | 0/9 [00:00<?, ?it/s]

In [9]:
table

,P1 (%),Space Group chi^2,N atoms EMD
WyFormer,2.04994,0.058459,8.845024
FlowMM,40.690097,12.781056,0.492154
MiAD,20.139697,4.568208,1.675489
DiffCSP++,1.843732,0.067514,8.67499
DiffCSP,31.566641,6.843615,0.63917
SymmCD,2.177203,0.070302,7.698487
MP-20-train,NaN,NaN,NaN
MP-20-test,1.802699,0.00001,0.002511
MP-20-val,NaN,NaN,NaN
